In [ ]:
# # Force install into the active Jupyter environment (This guarantees it installs tensorflow-addons into the exact Python environment your notebook is using.)
# import sys
# !{sys.executable} -m pip install tensorflow-addons
# !{sys.executable} -m pip install tensorflow
# !{sys.executable} -m pip install numpy
# !{sys.executable} -m pip install matplotlib
# !{sys.executable} -m pip install scikit-learn

# # successfully installed packages 

# !{sys.executable} -m pip install keras==2.1 # installing the version mentioned in the git repo 

In [ ]:
import tensorflow_addons as tfa
from tensorflow.keras.callbacks import ReduceLROnPlateau
import time
from models import models as new_models
from utils import *
import keras

from keras_unet_collection import models, utils
import matplotlib.pyplot as plt
print('TensorFlow {}; Keras {}'.format(tf.__version__, keras.__version__))

## Constants, Parameters and Datasets

In [ ]:
grid_size = 1024                # Set the size of the full input grid (likely 1024x1024 pixels)
AUTOTUNE = tf.data.AUTOTUNE     # Set TensorFlow's automatic tuning for dataset loading (optimizes performance)
batch_size = 2                  # Set the batch size (number of samples per training step)
split = 1                       # Choose which data split to use (1, 2, or 3 as available)
output_1d = False               # Whether the output labels should be flattened into 1D or kept 2D (here: 2D output)
EPOCHS = 200                    # Number of training epochs (complete passes over the training dataset)


# MODEL_NAME = "Attn-Unet-pretrained"
# LR = 0.00015
# MIN_LR = LR/5
# WARMUP_EPOCHS = 20

MODEL_NAME = "SWIN-Unet"        # Name of the model architecture to use
LR = 0.00018                    # Set initial learning rate for the optimizer
MIN_LR = LR/6                   # Set the minimum learning rate for learning rate decay                 
WARMUP_EPOCHS = 20              # Number of epochs to spend "warming up" before reaching the initial LR

# MODEL_NAME = "CASPIAN"
# LR = 0.0008
# MIN_LR = LR/10
# WARMUP_EPOCHS = 20


###### For Ablation studies ###########################

MODEL_NAME = "CASPIAN_beta_v4"
LR = 0.0008
MIN_LR = LR/10
WARMUP_EPOCHS = 20
version = 4
###### For Ablation studies ###########################

# ---------------------------------------------------------
# Data Loading Stage
# ---------------------------------------------------------


# If the output is not 1D, load and process a flood mask (used for post-processing model outputs)
if not output_1d:
    #Loading and processing the mask
    the_mask = get_the_mask()


# Load and preprocess the training and validation datasets
ds = {
    'train': tf.data.Dataset.load("./data/train_ds_aug_split_%d" % split).map(lambda f,x,y,yf: tf.py_function(clear_ds,
                                           inp=[f,x,y,yf, output_1d],
                                           Tout=[tf.float32, tf.float32])),
    'val': tf.data.Dataset.load("./data/val_ds_aug_split_%d" % split).map(lambda f,x,y,yf: tf.py_function(clear_ds,
                                           inp=[f,x,y,yf, output_1d],
                                           Tout=[tf.float32, tf.float32]))
}
# at the end -> both ds['train'] and ds['val'] are the training and validation datasets cleaned and ready to use


# If the model is an Attention U-Net, adjust the input channels accordingly
if "Attn" in MODEL_NAME:
    ds['train'] = ds['train'].map(lambda x,y: expand_input_channels(x, y))
    ds['val'] = ds['val'].map(lambda x,y: expand_input_channels(x, y))

# Print the number of elements in the training set
print("Size of the Training Dataset: %d  " % tf.data.experimental.cardinality(ds["train"]).numpy())

# Print the number of elements in the validation set
print("Size of the Validation Dataset: %d  " % tf.data.experimental.cardinality(ds["val"]).numpy())

# Take one sample from the training set and print its input shape and unique input values
input_sample, label_sample = next(iter(ds["train"]))
print("Training input shape: %s" % str(input_sample.numpy().shape))
print("Training input values: %s" % str(np.unique(input_sample.numpy())))

# Print the corresponding output (label) shape
print("Training output shape: %s " % str(label_sample.numpy().shape))

# Repeat the same for the validation set
input_sample, label_sample = next(iter(ds["val"]))
print("Validation input shape: %s" % str(input_sample.numpy().shape))
print("Validation output shape: %s " % str(label_sample.numpy().shape))


# -----------------------------------------
# Plot samples from the training set
# -----------------------------------------

# Number of samples to visualize
samples = 2

# Create a figure with 2 rows and 2 columns
fig, axs = plt.subplots(samples,2, facecolor='w', edgecolor='k')

# Adjust spacing between subplots
#fig.subplots_adjust(hspace = .2, wspace=.13)
fig.tight_layout(pad=2, h_pad=2.5, w_pad=2.5)

# Flatten the axes array for easier iteration
axs = axs.ravel()

# Initialize index for subplots
i = 0

# Loop through two samples from the training dataset
for element in list(ds['train'].as_numpy_iterator())[:samples]:
    x,y = element
    axs[i].imshow(x, origin="lower", interpolation='none', aspect='auto')
    axs[i].set_title("Sample input")
    i += 1
    axs[i].imshow(y, origin="lower", interpolation='none', aspect='auto')
    axs[i].set_title("Sample output")
    i += 1


# -----------------------------------------
# Prepare datasets for training
# -----------------------------------------

# Batch the datasets (combine samples into batches of size batch_size)
ds['train'] = ds['train'].batch(batch_size)
ds['val'] = ds['val'].batch(batch_size)

# Create a copy of the flood mask for each sample in the batch
the_mask = np.array([the_mask]*batch_size)

# Print the shape of the batched mask
print ("Shape of the mask: %s" % str(the_mask.shape))

In [ ]:
name = MODEL_NAME+"_split_{}_{}".format(str(split), time.strftime("%Y%m%d-%H%M%S"))

#TF Callbacks
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='models/logs/{}'.format(name), histogram_freq=1)

checkpoint = tf.keras.callbacks.ModelCheckpoint("./models/trained_models/%s/initial/" % MODEL_NAME,
                    monitor="val_loss", mode="min", save_weights_only=True,
                    save_best_only=True, verbose=1)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = EPOCHS//5)

steps_per_epoch = int(ds["train"].cardinality())#//batch_size
lr_schedule = LinearDecayPerEpoch(LR, steps_per_epoch, EPOCHS, MIN_LR)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.85, patience=10, min_lr=MIN_LR)
first_decay_steps = steps_per_epoch*(EPOCHS//3)
cosine_lr_restarts = tf.keras.optimizers.schedules.CosineDecayRestarts(LR,first_decay_steps, t_mul=1.0, m_mul=0.8)

# Compute the number of warmup batches
warmup_batches = WARMUP_EPOCHS * steps_per_epoch
# Create the Learning rate scheduler
warm_up_lr = WarmUpLearningRateScheduler(warmup_batches, init_lr=LR)

# Model Fitting
opt = tf.keras.optimizers.Adam(learning_rate=LR)
#opt = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
#opt = tfa.optimizers.AdamW(learning_rate=init_lr, weight_decay=0.0005)

if "CASP" in MODEL_NAME:
    if "_" not in MODEL_NAME:
        model = new_models.CASPIAN(input_shape=(grid_size, grid_size, 1), filters=72, input_centering=True,
                        depth=4, cardinality=34, activation='tanh', bottleneck_depth=8,
                        init="glorot_normal", sup_level=1, compression_factor= 0.85, gr_level= 4)
    else:
        model = new_models.CASPIAN_beta(input_shape=(grid_size, grid_size, 1), filters=72, input_centering=True,
                depth=4, cardinality=34, activation='tanh', bottleneck_depth=8,
                init="glorot_normal", sup_level=1, compression_factor= 0.85, gr_level= 4, version=version)
elif "SWIN" in MODEL_NAME:
    set_seed()
    model = new_models.Swin_unet(
        filter_num_begin = 64,
        depth = 4,
        stack_num_down = 2,
        stack_num_up = 2,
        patch_size = 8,
        att_heads = 4,
        input_centering=True,
        dropout = 0.0)
else:
    if "pretrained" in MODEL_NAME:
        model = models.att_unet_2d((1024, 1024, 3), filter_num=[32, 64, 128, 256], n_labels=1,
                               stack_num_down=2, stack_num_up=2, activation='ReLU',
                               atten_activation='ReLU', attention='add', output_activation='ReLU',
                               batch_norm=False, pool=True, unpool=False,
                               backbone='VGG19', weights='imagenet',
                               freeze_backbone=False, freeze_batch_norm=False,
                               name='attunet')
    else:
        model = models.att_unet_2d((1024, 1024, 3), filter_num=[32, 64, 128, 256], n_labels=1,
                               stack_num_down=2, stack_num_up=2, activation='ReLU',
                               atten_activation='ReLU', attention='add', output_activation='ReLU',
                               batch_norm=False, pool=True, unpool=False,
                               backbone='VGG19', weights=None,
                               freeze_backbone=False, freeze_batch_norm=False,
                               name='attunet')


if output_1d:
    model.compile(loss=tf.keras.losses.Huber(delta=0.5), optimizer=opt, metrics=[tf.keras.metrics.RootMeanSquaredError(), "mae"])
else:
    model.compile(loss=custom_loss(mask=the_mask), optimizer=opt, metrics=[custom_rmse(mask=the_mask), custom_mae(mask=the_mask)])

# Print a summary of the model architecture (layers, parameters, etc.)
model.summary()


# -----------------------------------------
# First Training Phase: Warmup
# -----------------------------------------

# Train the model for the full number of epochs (full training)
# Continue training from the warmup weights
# Callbacks here:
# - checkpoint: keep saving good models
# - tensorboard_callback: keep logging metrics
# - early_stop: stop training early if validation loss stops improving
# - reduce_lr: lower the learning rate when plateau is detected

history_warmup = model.fit(ds['train'],
                epochs=WARMUP_EPOCHS,
                validation_data=ds['val'],
                callbacks=[checkpoint, tensorboard_callback, warm_up_lr]) #PrintLearningRate()#reduce_lr#early_stop


# -----------------------------------------
# After Warmup Phase: Load Best Warmup Weights
# -----------------------------------------


# Reload the model weights saved after the warmup phase.
# This ensures training continues from the best model checkpoint after warmup (instead of random weights).
model.load_weights("./models/trained_models/%s/initial/" % MODEL_NAME)


# -----------------------------------------
# Second Training Phase: Full Training
# -----------------------------------------


# Continue training the model starting from the loaded warmup weights.
# Now train for the full number of epochs defined by EPOCHS.
# Use callbacks:
# - checkpoint: continue saving best models
# - tensorboard_callback: continue logging metrics
# - early_stop: stop training early if validation loss stops improving (avoids overfitting)
# - reduce_lr: reduce the learning rate if validation loss plateaus
history = model.fit(ds['train'],
                epochs=EPOCHS,
                validation_data=ds['val'],
                callbacks=[checkpoint, tensorboard_callback, early_stop, reduce_lr]) #PrintLearningRate()#reduce_lr#early_stop

# -----------------------------------------
# Finalizing: Load Best Full Training Weights
# -----------------------------------------

# Load again the best model weights obtained during the full training phase.
# This ensures the model saved in the next step is the best-performing version (lowest validation loss).
model.load_weights("./models/trained_models/%s/initial/" % MODEL_NAME)


# -----------------------------------------
# Saving the Fully Trained Model
# -----------------------------------------


# Save the fully trained model to disk in HDF5 (.h5) format.
# The filename includes the MODEL_NAME and the data split used (e.g., split 1, 2, or 3).
model.save("./models/trained_models/"+MODEL_NAME+"_split_{}".format(str(split)), save_format='h5')